# To do:
* Scrape listings
* List of Boroughs
* Data on crime rates, salaries etc... (Kaggle, Google, Gov stats)
* Save it all to database

In [120]:
borough = 'lambeth'
page = '1'
base_url = f'https://www.zoopla.co.uk/for-sale/property/{borough}/?page_size=25&q=lambeth&radius=0&results_sort=newest_listings&pn={page}'
print(base_url)

https://www.zoopla.co.uk/for-sale/property/lambeth/?page_size=25&q=lambeth&radius=0&results_sort=newest_listings&pn=1


In [121]:
from bs4 import BeautifulSoup
import requests

In [122]:
data = requests.get(base_url).content

In [123]:
soup = BeautifulSoup(data, 'html.parser')

In [124]:
listings_container = soup.find('div', {'class': 'css-kdnpqc-ListingsContainer'})

In [125]:
listings = listings_container.find_all('div', {'class': 'e2uk8e18'})

In [126]:
price = listings[0].find('p', {'class': 'eczcs4p0'}).getText()

In [147]:
# Solution 1
def features(feat):
    if feat:
        print(feat.find_next('p').getText())
    else:
        print(None)

feat_list = ['bed', 'bath', 'chair']
for i in range(len(feat_list)):
    features(listings[0].find('span', {'data-testid': feat_list[i]}))
    

2
2
1


In [146]:
# feat.attrs['data-testid'] == 'bed'
# feat.find_next('p').getText()